In [1]:
import os

In [2]:
%pwd

'/Users/poonamkumari/End-to-End-Chest-Classification-using-mlflow-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/poonamkumari/End-to-End-Chest-Classification-using-mlflow-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path
 
@dataclass(frozen=True)
class DataIngestionConfig:
    """Configuration for data ingestion process."""
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
       
        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Fetch data from the URL specified in the config.
        """
        try:
            dataset_url = self.config.source_url
            zip_download_path = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_path}")

            file_id = dataset_url.split("/")[-2]
            prefix= f"https://drive.google.com/uc?export=download&id="
            gdown.download(prefix+file_id, zip_download_path)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_path}")
            #return zip_download_path
        except Exception as e:
            logger.error(f"Failed to download data: {e}")
            raise

    def extract_zip_file(self) -> None:
        """
        Extracts the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    print("Data ingestion completed successfully!")
except Exception as e:
    print(f"Error occurred: {e}")
    raise

[2025-07-14 19:06:37,114: INFO: common]: YAML file loaded successfully from: config/config.yaml
[2025-07-14 19:06:37,118: INFO: common]: YAML file loaded successfully from: params.yaml
[2025-07-14 19:06:37,130: INFO: common]: Created directory at: artifacts
[2025-07-14 19:06:37,157: INFO: common]: Created directory at: artifacts/data_ingestion
[2025-07-14 19:06:37,158: INFO: 1943611322]: Downloading data from https://drive.google.com/file/d/1aHAJzlYitGE2bFPJURIgDXEKKmM5IA9M/view?usp=sharing into file artifacts/data_ingestion/data.zip


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1aHAJzlYitGE2bFPJURIgDXEKKmM5IA9M
From (redirected): https://drive.google.com/uc?export=download&id=1aHAJzlYitGE2bFPJURIgDXEKKmM5IA9M&confirm=t&uuid=4e135693-fe4a-4cdd-976f-cb8d23c69619
To: /Users/poonamkumari/End-to-End-Chest-Classification-using-mlflow-DVC/artifacts/data_ingestion/data.zip
100%|██████████| 84.3M/84.3M [00:15<00:00, 5.33MB/s]

[2025-07-14 19:06:56,402: INFO: 1943611322]: Downloaded data from https://drive.google.com/file/d/1aHAJzlYitGE2bFPJURIgDXEKKmM5IA9M/view?usp=sharing into file artifacts/data_ingestion/data.zip


Data ingestion completed successfully!
